In [1]:
import sys
sys.path.extend([ '../', '../../'])
import torch
import dataloader as dl
from args import Args
from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from train2 import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.imdb()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
data = torch.load('../data/data.pt')
print(data)

HeteroData(
  m={
    num_nodes=4666,
    y=[4666]
  },
  a={ num_nodes=5845 },
  d={ num_nodes=2271 },
  m_text_embed={ x=[4666, 128] },
  m_net_embed={ x=[4666, 128] },
  m_a_net_embed={ x=[4666, 128] },
  m_d_net_embed={ x=[4666, 128] },
  a_net_embed={ x=[5845, 128] },
  a_text_embed={ x=[5845, 128] },
  d_net_embed={ x=[2271, 128] },
  d_text_embed={ x=[2271, 128] },
  (m, walk, m)={ edge_index=[2, 32862] },
  (m, walk, a)={ edge_index=[2, 29894] },
  (m, walk, d)={ edge_index=[2, 13656] },
  (a, walk, m)={ edge_index=[2, 26087] },
  (a, walk, a)={ edge_index=[2, 35045] },
  (a, walk, d)={ edge_index=[2, 15898] },
  (d, walk, m)={ edge_index=[2, 11136] },
  (d, walk, a)={ edge_index=[2, 14009] },
  (d, walk, d)={ edge_index=[2, 6256] }
)


In [ ]:
print(data['m','walk','a'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['m_text_embed']['x'] = data['m_text_embed']['x'].to(device)
data['m_net_embed']['x'] = data['m_net_embed']['x'].to(device)
data['m_a_net_embed']['x'] = data['m_a_net_embed']['x'].to(device)
data['m_d_net_embed']['x'] = data['m_d_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['d_net_embed']['x'] = data['d_net_embed']['x'].to(device)
data['d_text_embed']['x'] = data['d_text_embed']['x'].to(device)

# Send all y tensors to the device
data['m']['y'] = data['m']['y'].to(device)

# Send all edge_index tensors to the device
data['m', 'walk', 'm']['edge_index'] = data['m', 'walk', 'm']['edge_index'].to(device)
data['m', 'walk', 'a']['edge_index'] = data['m', 'walk', 'a']['edge_index'].to(device)
data['m', 'walk', 'd']['edge_index'] = data['m', 'walk', 'd']['edge_index'].to(device)
data['a', 'walk', 'm']['edge_index'] = data['a', 'walk', 'm']['edge_index'].to(device)
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'd']['edge_index'] = data['a', 'walk', 'd']['edge_index'].to(device)
data['d', 'walk', 'm']['edge_index'] = data['d', 'walk', 'm']['edge_index'].to(device)
data['d', 'walk', 'a']['edge_index'] = data['d', 'walk', 'a']['edge_index'].to(device)
data['d', 'walk', 'd']['edge_index'] = data['d', 'walk', 'd']['edge_index'].to(device)

edge_indices = [ data['m', 'walk', 'm'].edge_index, data['m', 'walk', 'a'].edge_index, data['m', 'walk', 'd'].edge_index ]

In [ ]:
c_train_num = dl.train_num(data['m'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['m'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

In [ ]:
encoder1 = Het_ConEn(args.embed_dim, args.dropout)
encoder2 = Het_NetEn(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_m = EdgePredictor(args.embed_dim)
decoder_a = EdgePredictor(args.embed_dim)
decoder_d = EdgePredictor(args.embed_dim)
# decoder_m = torch.load('pretrained/decoder_m.pth')
# decoder_a = torch.load('pretrained/decoder_a.pth')
# decoder_d = torch.load('pretrained/decoder_d.pth')
decoder_list = [decoder_m, decoder_a, decoder_d]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [ ]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = '', train_mode = 'em_smote')

In [ ]:
# test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'gsm', train_mode = 'preT')

In [ ]:
# torch.save(decoder_list[0], 'pretrained/decoder_m.pth')
# torch.save(decoder_list[1], 'pretrained/decoder_a.pth')
# torch.save(decoder_list[2], 'pretrained/decoder_d.pth')
# torch.save(encoder1, 'pretrained/encoder1.pth')
# torch.save(encoder2, 'pretrained/encoder2.pth')